# ETL Analysis - BMW Sales Data

Este notebook analisa os dados do BMW e depura o processo ETL.


In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Add src to path
sys.path.append(os.path.join('..', 'src'))

# Import our modules
from etl.data_processor import DataProcessor
from etl.kaggle_extractor import KaggleExtractor
from database.loader import DatabaseLoader
from config.database import test_connection

print("✅ Imports successful!")


c:\Users\Rodrigo\anaconda3\envs\ai_engineering\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports successful!


In [2]:
# Load and examine the BMW data
df = pd.read_csv('../input/BMW sales data (2010-2024) (1).csv')

print("📊 Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Data types:\n{df.dtypes}")
print("\n📋 Sample data:")
df.head()


📊 Dataset Info:
Shape: (50000, 11)
Columns: ['Model', 'Year', 'Region', 'Color', 'Fuel_Type', 'Transmission', 'Engine_Size_L', 'Mileage_KM', 'Price_USD', 'Sales_Volume', 'Sales_Classification']
Data types:
Model                    object
Year                      int64
Region                   object
Color                    object
Fuel_Type                object
Transmission             object
Engine_Size_L           float64
Mileage_KM                int64
Price_USD                 int64
Sales_Volume              int64
Sales_Classification     object
dtype: object

📋 Sample data:


,Model,Year,Region,Color,Fuel_Type,Transmission,Engine_Size_L,Mileage_KM,Price_USD,Sales_Volume,Sales_Classification
0,5 Series,2016,Asia,Red,Petrol,Manual,3.5,151748,98740,8300,High
1,i8,2013,North America,Red,Hybrid,Automatic,1.6,121671,79219,3428,Low
2,5 Series,2022,North America,Blue,Petrol,Automatic,4.5,10991,113265,6994,Low
3,X3,2024,Middle East,Blue,Petrol,Automatic,1.7,27255,60971,4047,Low
4,7 Series,2020,South America,Black,Diesel,Manual,2.1,122131,49898,3080,Low


In [3]:
# Test data processing step by step
processor = DataProcessor()

print("🧹 Step 1: Data Cleaning")
cleaned_df = processor.clean_data(df)
print(f"Cleaned shape: {cleaned_df.shape}")

print("\n🔄 Step 2: Data Transformation")
try:
    transformed_df = processor.transform_bmw_data(cleaned_df)
    print(f"Transformed shape: {transformed_df.shape}")
    print(f"New columns: {set(transformed_df.columns) - set(cleaned_df.columns)}")
    print("\n✅ Transformation successful!")
except Exception as e:
    print(f"❌ Transformation failed: {e}")
    transformed_df = None


INFO:etl.data_processor:Starting data cleaning. Original shape: (50000, 11)
INFO:etl.data_processor:Dropped 0 duplicate rows
INFO:etl.data_processor:Dropped 0 rows with missing values
INFO:etl.data_processor:Data cleaning completed. Final shape: (50000, 11)
INFO:etl.data_processor:Starting BMW data transformation
INFO:etl.data_processor:BMW data transformation completed


🧹 Step 1: Data Cleaning
Cleaned shape: (50000, 11)

🔄 Step 2: Data Transformation
Transformed shape: (50000, 13)
New columns: {'year', 'fuel_type', 'color', 'sales_classification', 'sales_volume', 'price_usd', 'model', 'year_month', 'engine_size_l', 'mileage_km', 'total_sales', 'region', 'transmission'}

✅ Transformation successful!


In [4]:
# Examine the transformed data
if transformed_df is not None:
    print("📊 Transformed Data Info:")
    print(f"Shape: {transformed_df.shape}")
    print(f"Columns: {transformed_df.columns.tolist()}")
    print(f"Data types:\n{transformed_df.dtypes}")
    
    print("\n📋 Sample transformed data:")
    transformed_df.head()
    
    # Check for any sales-related columns
    sales_cols = [col for col in transformed_df.columns if 'sales' in col.lower()]
    print(f"\n💰 Sales-related columns: {sales_cols}")
    
    if sales_cols:
        print(f"Sample values from sales columns:")
        for col in sales_cols:
            print(f"{col}: {transformed_df[col].head().tolist()}")
else:
    print("❌ No transformed data available")


📊 Transformed Data Info:
Shape: (50000, 13)
Columns: ['model', 'year', 'region', 'color', 'fuel_type', 'transmission', 'engine_size_l', 'mileage_km', 'price_usd', 'sales_volume', 'sales_classification', 'year_month', 'total_sales']
Data types:
model                           object
year                    datetime64[ns]
region                          object
color                           object
fuel_type                       object
transmission                    object
engine_size_l                  float64
mileage_km                       int64
price_usd                        int64
sales_volume                     int64
sales_classification            object
year_month                      object
total_sales                    float64
dtype: object

📋 Sample transformed data:

💰 Sales-related columns: ['sales_volume', 'sales_classification', 'total_sales']
Sample values from sales columns:
sales_volume: [8300, 3428, 6994, 4047, 3080]
sales_classification: ['High', 'Low', 'Low', '

In [7]:
# Test database connection
print("🔌 Testing database connection...")
if test_connection():
    print("✅ Database connection successful!")
    
    # Test database loader
    print("\n📦 Testing database loader...")
    loader = DatabaseLoader()
    
    # Create tables
    print("Creating tables...")
    loader.create_tables()
    print("✅ Tables created successfully!")
    
    # Get database stats
    stats = loader.get_database_stats()
    print(f"\n📊 Database stats: {stats}")
    
else:
    print("❌ Database connection failed!")


🔌 Testing database connection...
Database connection failed: 'utf-8' codec can't decode byte 0xe7 in position 78: invalid continuation byte
❌ Database connection failed!


In [ ]:
# Test loading data into database
if transformed_df is not None and 'loader' in locals():
    print("📥 Testing data loading...")
    
    # Take a small sample for testing
    sample_df = transformed_df.head(100)
    print(f"Loading {len(sample_df)} records...")
    
    try:
        success = loader.load_bmw_sales(sample_df, "BMW Test Data")
        if success:
            print("✅ Data loaded successfully!")
            
            # Check database stats again
            stats = loader.get_database_stats()
            print(f"\n📊 Updated database stats:")
            for table, info in stats.items():
                print(f"  {table}: {info.get('row_count', 0)} rows")
        else:
            print("❌ Data loading failed!")
    except Exception as e:
        print(f"❌ Error loading data: {e}")
else:
    print("❌ Cannot test data loading - missing transformed data or loader")


In [ ]:
# Query the database to see what was loaded
if 'loader' in locals():
    print("🔍 Querying database...")
    
    try:
        # Query BMW sales data
        query = "SELECT * FROM bmw_sales LIMIT 5"
        results = loader.execute_query(query)
        
        if results:
            print("✅ BMW Sales data found:")
            for i, row in enumerate(results):
                print(f"  Row {i+1}: {row}")
        else:
            print("❌ No BMW sales data found")
            
        # Query data sources
        query = "SELECT * FROM data_sources"
        sources = loader.execute_query(query)
        
        if sources:
            print(f"\n✅ Data sources found: {len(sources)}")
            for source in sources:
                print(f"  {source}")
        else:
            print("\n❌ No data sources found")
            
    except Exception as e:
        print(f"❌ Error querying database: {e}")
else:
    print("❌ No database loader available")


## Resumo da Análise

Este notebook testa cada etapa do processo ETL:

1. **Carregamento de dados**: Lê o CSV do BMW
2. **Limpeza de dados**: Remove duplicatas e valores faltantes
3. **Transformação**: Cria colunas derivadas e padroniza dados
4. **Conexão com banco**: Testa a conexão PostgreSQL
5. **Criação de tabelas**: Cria as tabelas necessárias
6. **Carregamento**: Insere dados no banco
7. **Consulta**: Verifica se os dados foram inseridos corretamente

Execute as células acima para depurar o processo ETL passo a passo.
